In [ ]:
%load_ext blackcellmagic

#### Imports and setup

In [ ]:
import numpy as np
import pandas as pd
import deepdish as dd
import h5py
import nibabel.freesurfer.mghformat as mgh
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.anova import AnovaRM

In [ ]:
from spacestream.core.constants import SUBJECTS, CORE_ROI_NAMES
from spacestream.core.paths import DATA_PATH, RESULTS_PATH
from spacestream.utils.general_utils import sem
from spacestream.utils.get_utils import get_mapping

In [ ]:
seeds = [0,1,2,3,4]
hemis = ["lh","rh"]
sw_list = [
            "0.0",
            "0.1",
            "0.25",
            "0.5",
            "1.25",
            "2.5",
            "25.0",
        ]
sup_case = ["simCLR", "supervised"]
colors = {}
colors["simCLR"], colors["supervised"] = "#720298", "#B59410"

### Top panel - distance similarity (r)

#### Load and format data

In [ ]:
data = np.zeros((len(sup_case), len(sw_list), len(seeds)))
full_results = {
    "Version": [],
    "Spatial_Weight": [],
    "Stream": [],
    "Seed": [],
    "Subject": [],
    "Hemi": [],
    "Corr": [],
}
for hemi in hemis:
    for sidx, seed in enumerate(seeds):
        for supx, sup_str in enumerate(sup_case):
            smooth_path = (
                RESULTS_PATH
                + "analyses/spatial/TDANNs/"
                + "smoothness_calc_by_stream_"
                + ("lh_" if hemi == "lh" else "")
                + "unit2voxel"
                + ("_supervised" if sup_str == "supervised" else "")
                + (("_seed" + str(seed)) if seed > 0 else "")
                + "_correlations_by_unit_ckptfinal.hdf"
            )

            u2v_smoothness = dd.io.load(smooth_path)
            for i in range(len(sw_list)):
                for j in range(len(SUBJECTS)):

                    for streamx, stream in enumerate(CORE_ROI_NAMES):
                        full_results["Version"].append(str(supx))
                        full_results["Seed"].append(str(sidx))
                        full_results["Hemi"].append(hemi)
                        full_results["Spatial_Weight"].append(float(sw_list[i]))
                        full_results["Stream"].append(stream)
                        full_results["Subject"].append(SUBJECTS[j])
                        full_results["Corr"].append(
                            u2v_smoothness["thirddist_r"][i, j, streamx]
                        )

In [ ]:
df = pd.DataFrame(full_results)

In [ ]:
s2s_data = np.zeros((8,7,3,len(hemis), len(seeds)))

for hidx, hemi in enumerate(hemis):
    for sidx, seed in enumerate(seeds):
        smooth_path = (
                    RESULTS_PATH
                    + "analyses/spatial/brains/"
                    + "smoothness_calc_by_stream_"
                    + ("lh_" if hemi == "lh" else "")
                    + "voxel2voxel"
                    + (("_seed" + str(seed)) if seed > 0 else "")
                    + "_correlations_by_unit_ckptfinal.hdf"
                )
        v2v_smoothness = dd.io.load(smooth_path)
        s2s_data[:,:,:,hidx,sidx] = v2v_smoothness["thirddist_r"]
        
del v2v_smoothness

s2s = np.mean(np.mean(s2s_data, axis=-1), axis=-1)

In [ ]:
df

In [ ]:
df.to_csv('/share/kalanit/users/dfinzi/Desktop/processed_spacestream_csvs/F03_B.csv', index=False)  

In [ ]:
np.mean(s2s,axis=1)

#### Plotting

In [ ]:
# log plot version
sns.set_theme(style="ticks")

line_kwargs = {
    "marker": ".",
    "markersize": 25,
    "lw": 1,
    "markeredgecolor": "w",
    "markeredgewidth": 2,
}

for streamx, stream in enumerate(CORE_ROI_NAMES):
    fig, ax = plt.subplots(figsize=(3, 5))
    sns.lineplot(
        df[df["Stream"].isin([stream])],
        x="Spatial_Weight",
        y="Corr",
        hue="Version",
        palette=[colors["simCLR"], colors["supervised"]],
        errorbar="se",
        **line_kwargs
    )
    ax.set_xscale("symlog", linthresh=0.09)
    ax.set_xlim([-0.01, 50])
    ax.set_xticks([], minor=True)
    ax.set_xticks([0, 0.1, 0.25, 0.5, 1.25, 2.5, 25])
    ax.set_xticklabels([0, 0.1, "", "", 1.25, "", 25])

    y1 = np.mean(np.mean(np.squeeze(s2s[:, :, streamx]), axis=1)) + np.std(
        np.mean(np.squeeze(s2s[:, :, streamx]), axis=1)
    )
    y2 = np.mean(np.mean(np.squeeze(s2s[:, :, streamx]), axis=1)) - np.std(
        np.mean(np.squeeze(s2s[:, :, streamx]), axis=1)
    )
    ax.axhspan(y1, y2, xmin=0, xmax=1, color="lightgray", alpha=0.75)

    h, l = ax.get_legend_handles_labels()
    if streamx == 2:
        ax.legend(h, ["Self-supervised", "Supervised"])
    else:
        ax.legend([], [], frameon=False)

    ax.set_xlabel("Spatial Weight", fontsize=20)
    ax.set_ylabel("Correlation", fontsize=20)
    ax.set_yticks([])
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)

    plt.ylim(bottom=0.0, top=0.5)
    plt.show()

    save_path = RESULTS_PATH + "/figures/3b_top_" + stream + "_" + "_despined_SD.png"
    fig.savefig(save_path, dpi=100)

#### Statistics

In [ ]:
dorsal = df[df["Stream"].isin(["Dorsal"])]
lateral = df[df["Stream"].isin(["Lateral"])]
ventral = df[df["Stream"].isin(["Ventral"])]

In [ ]:
aovrm = AnovaRM(dorsal, 'Corr', 'Subject', within=['Spatial_Weight','Version'], aggregate_func='mean')
res = aovrm.fit()
print(res.anova_table.to_latex())

In [ ]:
aovrm = AnovaRM(lateral, 'Corr', 'Subject', within=['Spatial_Weight','Version'], aggregate_func='mean')
res = aovrm.fit()
print(res.anova_table.to_latex())

In [ ]:
aovrm = AnovaRM(ventral, 'Corr', 'Subject', within=['Spatial_Weight','Version'], aggregate_func='mean')
res = aovrm.fit()
print(res.anova_table.to_latex())

### Bottom panel - functional similarity (r)

#### Load and format data

In [ ]:
def correct_for_voxel_noise_ceiling(NC, mapping):

    brain_r = np.sqrt(
        NC[mapping["winning_idx"].astype(int)] / 100
    )  # convert from R^2 to r
    mapping["winning_roi"] = mapping["winning_roi"].astype(np.float32)

    if np.sum(np.isinf(mapping["winning_test_corr"])) > 0:
        mapping["winning_test_corr"][np.isinf(info["winning_test_corr"])] = np.nan

    corrected = mapping["winning_test_corr"] / brain_r
    corrected[mapping["winning_test_corr"] == 0] = np.nan

    return corrected

In [ ]:
full_results = {"Hemi": [], "Subject": [], "Stream": [], "Seed": [], "Version": [], "Spatial_Weight": [], "Corr": []}
corr_perf = np.empty(
    (
        len(hemis),
        len(SUBJECTS),
        len(CORE_ROI_NAMES),
        len(seeds),
        len(sup_case),
        len(sw_list),
    )
)
corr_perf[:] = np.nan

for hidx, hemi in enumerate(hemis):
    for sidx, subj in enumerate(SUBJECTS):

        # get ROI info
        mgh_file = mgh.load(DATA_PATH + "brains/" + hemi + ".ministreams.mgz")
        streams = mgh_file.get_fdata()[:, 0, 0].astype(int)
        # get noise ceiling estimates
        mgh_file = mgh.load(
            DATA_PATH + "brains/NC/subj" + subj + "/" + hemi + ".nc_3trials.mgh"
        )
        NC = mgh_file.get_fdata()[:, 0, 0]
        NC_trim = NC[streams != 0]
        NC_trim[NC_trim == 0] = np.nan  # Set all 0s to nans to avoid dividing by 0

        for vidx, version in enumerate(sup_case):

            if version == "supervised":
                v_str = version
                supervised = 1
            else:
                v_str = "self_supervised"
                supervised = 0

            for widx, weight in enumerate(sw_list):

                for midx, seed in enumerate(seeds):
                    mapping = get_mapping(
                        subj_name="subj" + str(subj),
                        spatial_weight=weight,
                        model_seed=seed,
                        supervised=supervised,
                        hemi=hemi,
                    )
                    corrected = correct_for_voxel_noise_ceiling(NC_trim, mapping)

                    for ridx, r in enumerate(CORE_ROI_NAMES):
                        corr_perf[hidx, sidx, ridx, midx, vidx, widx] = np.nanmean(
                            corrected[mapping["winning_roi"] == ridx + 5]
                        )
                        full_results["Version"].append(v_str)
                        full_results["Spatial_Weight"].append(float(weight))
                        full_results["Seed"].append(seed)
                        full_results["Subject"].append(subj)
                        full_results["Stream"].append(r)
                        full_results["Hemi"].append(hemi)
                        full_results["Corr"].append(
                            corr_perf[hidx, sidx, ridx, midx, vidx, widx]
                        )

In [ ]:
df = pd.DataFrame(full_results)
df

In [ ]:
df.to_csv('/share/kalanit/users/dfinzi/Desktop/processed_spacestream_csvs/F03_B_func.csv', index=False)  

In [ ]:
## load subject2subject estimates
s2s_corrected_by_stream= np.zeros((len(SUBJECTS),len(seeds),len(CORE_ROI_NAMES), len(hemis)))

for hidx, hemi in enumerate(hemis):
    
    for sidx, subj in enumerate(SUBJECTS):

        for seedix, seed in enumerate(seeds):

            load_path = (RESULTS_PATH
                            + "mappings/one_to_one/voxel2voxel/target_subj"
                            + subj
                            + "/mode_"
                            + hemi
                            + "_ministreams_HVA_only_radius5_max_iters100_constant_radius_2.0dist_cutoff_constant_dist_cutoff_spherical"
                            + ("_CV_seed" + str(seed))
                            + "_final_voxel2voxel_correlation_info.hdf5"
                        )
            with h5py.File(load_path, "r") as f:

                for r, ridx in enumerate(CORE_ROI_NAMES):
                    s2s_corrected_by_stream[sidx,seedix,r,hidx] =  np.nanmean(f['corrected_test_corr'][:][f['winning_roi'][:] == r+5])
across_seed_corrected_mean = np.mean(np.mean(s2s_corrected_by_stream,axis=-1),axis=1)

In [ ]:
across_seed_corrected_mean

#### Plotting

In [ ]:
for ridx, stream in enumerate(CORE_ROI_NAMES):
    fig, ax = plt.subplots(figsize = (3,5))

    sns.lineplot(
        df[df["Stream"].isin([stream])],
        x="Spatial_Weight",
        y="Corr",
        hue="Version",
        palette=[colors["simCLR"], colors["supervised"]],
        errorbar='se',
        **line_kwargs
    )
    ax.set_xscale("symlog", linthresh=0.09)
    ax.set_xlim([-0.01, 50])
    ax.set_xticks([], minor=True)
    ax.set_xticks([0, 0.1, 0.25, 0.5, 1.25, 2.5, 25])
    ax.set_xticklabels([0, 0.1, "", "", 1.25, "", 25])

    xmin = 0
    xmax = 1
    vals = across_seed_corrected_mean[:,ridx]
    y1 = np.mean(vals) - np.std(vals)
    y2 = np.mean(vals) + np.std(vals)
    ax.axhspan(y1, y2, xmin=xmin, xmax=xmax, color="lightgray", alpha=0.75)
    ax.legend([],[], frameon=False)

    ax.set_xlabel("Spatial Weight", fontsize=20)
    ax.set_ylabel("Correlation", fontsize=20)
    ax.set_yticks([])
    plt.ylim(bottom=0.0, top=0.5)

    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["top"].set_visible(False)


    plt.show()

    save_path = RESULTS_PATH + "/figures/3b_bottom_" + stream + "_" + "_despined_SD.png"
    fig.savefig(save_path, dpi=100)

#### Statistics

In [ ]:
dorsal_func = df[df["Stream"].isin(["Dorsal"])]
lateral_func = df[df["Stream"].isin(["Lateral"])]
ventral_func = df[df["Stream"].isin(["Ventral"])]

In [ ]:
aovrm = AnovaRM(dorsal_func, 'Corr', 'Subject', within=['Spatial_Weight','Version'], aggregate_func='mean')
res = aovrm.fit()
print(res.anova_table.to_latex())

In [ ]:
aovrm = AnovaRM(lateral_func, 'Corr', 'Subject', within=['Spatial_Weight','Version'], aggregate_func='mean')
res = aovrm.fit()
print(res.anova_table.to_latex())

In [ ]:
aovrm = AnovaRM(ventral_func, 'Corr', 'Subject', within=['Spatial_Weight','Version'], aggregate_func='mean')
res = aovrm.fit()
print(res.anova_table.to_latex())